In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from datetime import datetime
from tqdm import tqdm
from scipy.stats import poisson, skellam
from scipy.optimize import minimize
import statsmodels.api as sm
import statsmodels.formula.api as smf
from poisson_func import * 

plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (16,9) 

## Simulación

In [ ]:
new_sim = True
N_sim = 10000


df = pd.read_csv("matches_chile.csv")
df_tabla_2019 = pd.read_csv("Tabla2019.csv", index_col = "Equipo")
df_tabla_2019.sort_values(by=["PTS","DG"], ascending = False, inplace = True)
teams = df.Local.unique()
poisson_model = fit_poisson_model(df)

if new_sim == False:
    df_tabla_2020 = pd.read_csv("Tabla2020.csv", index_col = "Equipo")
    sim_poisson_local = pd.read_csv("sim_poisson_local.csv", index_col = [0,1])
    sim_poisson_local.columns = range(1, N_sim + 1)
    sim_poisson_visita = pd.read_csv("sim_poisson_visita.csv", index_col = [0,1])
    sim_poisson_visita.columns = range(1, N_sim + 1)
    df_posicion = pd.read_csv("df_posicion.csv", index_col = 0)
    df_desc_stats = pd.read_csv("df_desc_stats.csv", index_col = 0)
    summary_reasons = pd.read_excel("resumen_descenso.xlsx", index_col = 0)
    df_cd3 = pd.read_csv("df_cd3.csv", index_col = 0)
    df_rel_matches = pd.read_csv("df_rel_matches.csv", index_col = 0)
else:
    np.random.seed(42)
    #plot_poisson_dist(df)
    df_tabla_2020 = current_table(df, teams)
    df_posicion = pd.DataFrame(columns = ['Equipo', 'Tabla', 'n_sim', 'Posición'])
    N_iter = 10

    sim_poisson_local, sim_poisson_visita = poisson_tournament(df, poisson_model, N = N_sim)
    df_posicion = summary_positions(sim_poisson_local, sim_poisson_visita, N_sim, teams, df_tabla_2019,
                                        df_tabla_2020)
    df_desc_stats = relegation_stats(N_sim, df_posicion, poisson_model)
    summary_reasons = create_summary_reasons(df_desc_stats, N_sim)
    df_cd3, df_rel_matches = cases_distribution(df_posicion, N_sim)

76it [00:01, 71.04it/s]
  0%|          | 0/10000 [00:00<?, ?it/s]

0:00:02.340995


  9%|▉         | 892/10000 [09:36<1:42:26,  1.48it/s]

## Prob de campeonar

In [ ]:
df_posicion[(df_posicion["Posición"] == 1)&(df_posicion.Tabla == "Absoluta")].Equipo.value_counts()/N_sim

## Prob partido definición Colo Colo vs Universidad de Chile

In [ ]:
df_cc_uch = df_rel_matches[(df_rel_matches.team_1.isin(["Colo-Colo","Universidad de Chile"]))&
               (df_rel_matches.team_2.isin(["Colo-Colo","Universidad de Chile"]))]
print("Porcentaje de CC vs UCH:", 100*df_cc_uch.shape[0]/N_sim, "%")
df_cc_uch.head()

## Casos de descenso de UCH

#### Prob UCH último Absoluta

In [ ]:
df_ult_abs = df_posicion[(df_posicion.Tabla == "Absoluta")&
            (df_posicion["Posición"] == 18)&
            (df_posicion["Equipo"] == "Universidad de Chile")]
df_ult_abs.shape[0]/N_sim

In [ ]:
df_pen_abs = df_posicion[(df_posicion.Tabla == "Absoluta")&
            (df_posicion["Posición"] == 17)&
            (df_posicion["Equipo"] == "Universidad de Chile")]
df_pen_abs.shape[0]/N_sim

In [ ]:
mod_boolean = False
if mod_boolean == True:
    team = "Universidad de Chile"
    N_matches = 5
    goles_team = 0
    goles_rival = 1
    sim_poisson_local_mod, sim_poisson_visita_mod = sim_poisson_modification(team, N_matches, goles_team, 
                                                                             goles_rival, sim_poisson_local, 
                                                                             sim_poisson_visita)
    df_posicion_mod = summary_positions(sim_poisson_local_mod, sim_poisson_visita_mod, N_sim, teams, 
                                        df_tabla_2019, df_tabla_2020)
    df_desc_stats_mod = relegation_stats(N_sim, df_posicion_mod, poisson_model)
    summary_reasons_mod = summary_reasons(df_desc_stats_mod, N_sim)
    df_cd3_mod, df_rel_matches_mod = cases_distribution(df_posicion_mod, N_sim)

#### Prob UCH último Ponderada

In [ ]:
df_ult_pon = df_posicion[(df_posicion.Tabla == "Ponderada")&
            (df_posicion["Posición"] == 18)&
            (df_posicion["Equipo"] == "Universidad de Chile")]
sim_ult_pon = df_ult_pon.n_sim
df_ult_pon.shape[0]/N_sim

In [ ]:
df_pen_pon = df_posicion[(df_posicion.Tabla == "Ponderada")&
            (df_posicion["Posición"] == 17)&
            (df_posicion["Equipo"] == "Universidad de Chile")]
sim_pen_pon = df_pen_pon.n_sim
df_pen_pon.shape[0]/N_sim

In [ ]:
summary_reasons

In [ ]:
team = "Universidad de Chile"
probs_relegation(team, df_posicion, N_sim)

In [ ]:
team = "Iquique"
probs_relegation(team, df_posicion, N_sim)

In [ ]:
team = "Universidad de Concepción"
probs_relegation(team, df_posicion, N_sim)